In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train = pd.read_csv(f"/kaggle/input/feedback-prize-english-language-learning/train.csv")
test = pd.read_csv(f"/kaggle/input/feedback-prize-english-language-learning/test.csv")

In [3]:
train.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [4]:
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

In [6]:
train["text"] = train["full_text"].apply(lambda sentence: " ". join([stemmer.stem(x) for x in nltk.word_tokenize(sentence.lower()) if x not in stop_words]))
test["text"] = test["full_text"].apply(lambda sentence: " ". join([stemmer.stem(x) for x in nltk.word_tokenize(sentence.lower()) if x not in stop_words]))

In [7]:
train.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,text
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,"think student would benefit learn home , wont ..."
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,problem chang let best matter happen chang min...
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,"dear , princip u chang school polici grade b a..."
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,best time life becom . agre greatest accomplis...
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,small act kind impact peopl chang peopl becom ...


In [8]:
vectorizer = TfidfVectorizer()
vectorizer.fit(train['text'])

TfidfVectorizer()

In [9]:
X = vectorizer.transform(train['text'])

In [11]:
X

<3911x15146 sparse matrix of type '<class 'numpy.float64'>'
	with 358460 stored elements in Compressed Sparse Row format>

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, train['cohesion'], random_state=134)

In [14]:
X_train

<2933x15146 sparse matrix of type '<class 'numpy.float64'>'
	with 269010 stored elements in Compressed Sparse Row format>

In [15]:
y_train

1688    3.0
1948    4.0
1109    3.0
3243    3.0
3175    2.5
       ... 
114     2.5
1399    2.0
3448    2.5
1574    2.5
1507    3.0
Name: cohesion, Length: 2933, dtype: float64

In [16]:
from sklearn import metrics

In [17]:
from sklearn.neighbors import KNeighborsRegressor

In [21]:
r = KNeighborsRegressor(n_neighbors=10)
r.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=10)

In [22]:
y_pred = r.predict(X_test)
metrics.mean_absolute_error(y_pred, y_test)

0.5459611451942741

In [23]:
metrics.r2_score(y_pred, y_test)

-7.634459596810332

In [24]:
Xt = vectorizer.transform(test['text'])

In [25]:
r.predict(Xt)

array([2.85, 3.25, 3.3 ])